In [16]:
from dotenv import load_dotenv
import logging

# ==================== 配置日志 ====================
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
load_dotenv(override=True)

True

In [15]:
# ==================== 初始化模型 ====================
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(model="deepseek-chat", temperature=0.7)

LangGraph 内置了一个持久化层，通过检查点（checkpointer）机制，专门维护`同一个线程内`的历史消息。当你在编译图时传入一个检查点器，它会在每个“超级步骤”（super-step）结束时保存图状态的检查点。这些检查点会存储在特定的“线程”（thread）中，并可在图执行结束后进行访问。正因如此，执行完成后仍能访问图的状态，从而实现了多项强大功能，包括人机协同（human-in-the-loop）、记忆能力（memory）、时间回溯（time travel）以及容错性（fault-tolerance）。

<div align=center><img src="https://techinsight.com.cn/course/checkpoints.avif" width=60%></div>

1. Graph（图）
    - 含义：表示整个工作流的结构。
    - 内容：
        - Node 1 和 Node 2 是两个处理节点（如 LLM 调用、工具调用等）。
        - 箭头表示控制流：从 Node 1 → Node 2。
        - 初始状态：{state: "I"} → 经过 Node 1 后变为 "I heart" → 再经过 Node 2 变为 "I heart langgraph"。
2. Super-steps（超级步）
    - 含义：执行的基本单位，每个顺序节点对应一个独立的 super-step。
    - 规则：
        - 串行节点：每个节点是一个独立的 super-step。
        - 并行节点：多个并行节点共享同一个 super-step
    - 作用：将复杂的图分解为可调度的步骤。支持异步执行和并行处理。
    - 🔁 每个 super-step 都会触发一次状态更新和检查点生成。
3. Checkpoints（检查点）
    - 含义：在每个 super-step 结束时生成的状态快照。
    - 📦 检查点 = 当前状态 + 下一步指令 + 元数据
    - 内容：
        - 粉色框：Node 1 执行后的状态 {state: "I heart"} + 元数据（如 next: node 2）
        - 蓝色框：Node 2 执行后的状态 {state: "I heart langgraph"} + 元数据（如 next: end）
    - 作用：
        - 记录当前状态和执行上下文。
        - 支持中断恢复（可以从最近的检查点继续执行）。
        - 实现调试与回溯。

4. Thread（线程）
    - 含义：一组相关联的 checkpoints 的集合，是 LangGraph 中实现持久化和可追溯性的基础单元，也就是大家平时在用AI对话时候新建的会话。
    - 内容：
        - 用虚线框包裹两个 checkpoints，表示它们属于同一个 thread。
    - 作用：
        - 区分不同用户的会话或任务。
        - 支持多用户并发和上下文隔离。
5. StateSnapshot（状态快照）
    - LangGraph 的核心数据结构之一。
    - 含义：Checkpoint 的具体数据类型，包含当前状态、下一步指令和元数据。
    - 内容：
        - StateSnapshot是一个类，表示每个 checkpoint 的数据结构。
    - 作用：
        - 提供统一的数据接口，便于存储、序列化和传输。
        - 是实现状态持久化的核心。


# Thread

线程（thread）是由检查点器（checkpointer）为每个保存的检查点分配的一个唯一标识符（ID）。它包含了多次运行所累积的状态。当执行一次运行时，助手底层图的状态将被持久化到该线程中。检查点器使用 thread_id 作为存储和检索检查点的主键。如果没有提供 thread_id，检查点器将无法保存状态，也无法在中断后恢复执行，因为检查点器正是通过 thread_id 来加载已保存的状态的。

所以，在使用检查点器调用图时，你必须在配置（config）的 configurable 部分指定一个 thread_id，例如：

In [ ]:
{"configurable": {"thread_id": "1"}}

# Checkpoint

在某一特定时间点，线程的状态被称为检查点（checkpoint）。检查点是在每个超步（super-step）结束时保存的图状态快照，由 StateSnapshot 对象表示，包含以下关键属性：
- config：与此检查点关联的配置信息。
- metadata：与此检查点关联的元数据。
- values：此时各状态通道（state channels）的值。
- next：一个元组，包含图中下一步要执行的节点名称。
- tasks：一个包含 PregelTask 对象的元组，描述了接下来要执行的任务。
  - 如果该步骤之前曾尝试执行但失败，其中会包含错误信息；
  - 如果图在某个节点内部被动态中断（例如人为干预或条件中断），tasks 中还会包含与中断相关的附加数据。

## 基本用法

要使用检查点，除了需要在config中传入thread_id，还需要初始化一个Checkpointer对象。在底层，该功能由符合 BaseCheckpointSaver接口的检查点器（checkpointer）对象提供支持。LangGraph提供了多种检查点器实现，均通过独立、可安装的库来实现：
- langgraph-checkpoint：
    - 提供检查点器的基础接口（BaseCheckpointSaver）以及序列化/反序列化接（SerializerProtocol）。
    - 内置了用于实验和开发的内存检查点器实现（InMemorySaver）。LangGraph 默认已包含此库，无需额外安装。
- langgraph-checkpoint-sqlite
    - 基于 SQLite 数据库的 LangGraph 检查点器实现（SqliteSaver / AsyncSqliteSaver）。非常适合本地开发、实验和轻量级工作流。需单独安装。
- langgraph-checkpoint-postgres
    - 基于 PostgreSQL 数据库的高级检查点器实现（PostgresSaver / AsyncPostgresSaver），也是 LangSmith 内部所使用的实现。适用于生产环境，需单独安装。

为了便于演示其后续checkpoint方法的介绍，我们使用PostgresSaver来作为主要的检查点器。Postgresql数据库的安装网上有很多教程，这里不再赘述，大家自行找教程安装即可。

对应包的安装命令：
```bash
pip install langgraph-checkpoint-postgres
```

然后还需要提供DB_URI，可以将DB_URI添加到环境变量中：
```bash
export DB_URI=postgresql://<username>:<password>@localhost:5432/<db_name>?sslmode=disable
```

In [ ]:
# 内存检查点，仅在开发和测试中使用
# from langgraph.checkpoint.memory import InMemorySaver 
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables import RunnableConfig
from typing import Annotated
from typing_extensions import TypedDict
from operator import add
import os

db_uri = os.getenv("DB_URI")
if not db_uri:
    raise ValueError("DB_URI environment variable is not set")

class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]

def node_a(state: State):
    return {"foo": "a", "bar": ["a"]}

def node_b(state: State):
    return {"foo": "b", "bar": ["b"]}

with PostgresSaver.from_conn_string(db_uri) as checkpointer:
    checkpointer.setup()  # 自动创建相关表结构

    workflow = StateGraph(State)
    workflow.add_node(node_a)
    workflow.add_node(node_b)
    workflow.add_edge(START, "node_a")
    workflow.add_edge("node_a", "node_b")
    workflow.add_edge("node_b", END)

    graph = workflow.compile(checkpointer=checkpointer)

    config: RunnableConfig = {"configurable": {"thread_id": "example_thread_2"}}

    for chunk in graph.stream(
        {"foo": "", "bar":[]},
        config,  
        stream_mode="values",
    ):
        print(chunk)


{'foo': '', 'bar': ['a', 'b']}
{'foo': 'a', 'bar': ['a', 'b', 'a']}
{'foo': 'b', 'bar': ['a', 'b', 'a', 'b']}


<div align=center><img src="https://techinsight.com.cn/course/checpoints%E8%A1%A8%E7%BB%93%E6%9E%84.png" width=30%></div>

<div align=center><img src="https://techinsight.com.cn/course/checkponits%E8%A1%A8%E6%98%8E%E7%BB%86.png" width=100%></div>

## 读取检查点

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.postgres import PostgresSaver
from langchain.messages import HumanMessage
from langchain.agents import AgentState
import os

db_uri = os.getenv("DB_URI")
if not db_uri:
    raise ValueError("DB_URI environment variable is not set")

# 定义自定义状态类，验证存储是否是完整的state信息
class CustomState(AgentState):
    user_age: int

with PostgresSaver.from_conn_string(db_uri) as checkpointer:
    checkpointer.setup()  
    
    agent = create_agent(
        model,
        state_schema=CustomState,
        checkpointer=checkpointer,
        debug=True,
    )
    agent.invoke(
        input={"messages": [HumanMessage(content="你好，我叫Pet")], "user_age": 18},
        config={
            "configurable": {
                "thread_id": "example_thread_1"
            }
        }
    )

2026-02-27 18:53:20,619 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"


[values] {'messages': [HumanMessage(content='你好，我叫Pet', additional_kwargs={}, response_metadata={}, id='b1578147-3bb4-48dc-9cfd-175288da9fa8')], 'user_age': 18}
[updates] {'model': {'messages': [AIMessage(content='你好，Pet！很高兴认识你！😊\n\nPet是个很特别的名字呢！有什么我可以帮助你的吗？无论是聊天、回答问题，还是需要一些建议，我都很乐意为你提供帮助。今天过得怎么样？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 8, 'total_tokens': 53, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '3826d6d9-2371-4964-97a7-58517b4b934e', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c9ebb-4ac4-7b81-98eb-e7644e8799b1-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 8, 'output_tokens': 45, 'total_tokens': 53, 'input_token_detai

### 访问线程状态

#### get_tuple（checkpointer）

get_tuple是底层方法 ，负责从存储中取出原始检查点数据：
- 配置（config）中所提供的线程 ID 所关联的最新检查点；
- 或者，如果在 config 中同时提供了 checkpoint_id，则返回该线程下对应于该检查点 ID 的特定检查点的原始数据。

In [69]:
with PostgresSaver.from_conn_string(db_uri) as checkpointer:

    checkpoint = checkpointer.get_tuple(
        config={
            "configurable": {
                "thread_id": "example_thread_1",
                # "checkpoint_id": "",
            }
        }
    )
    print(checkpoint)
    # print(checkpoint.checkpoint["channel_values"])

CheckpointTuple(config={'configurable': {'thread_id': 'example_thread_1', 'checkpoint_ns': '', 'checkpoint_id': '1f113ca8-9525-6238-8001-91170acbc817'}}, checkpoint={'v': 4, 'id': '1f113ca8-9525-6238-8001-91170acbc817', 'ts': '2026-02-27T10:53:23.339078+00:00', 'versions_seen': {'model': {'branch:to:model': '00000000000000000000000000000002.0.5230344121653175'}, '__input__': {}, '__start__': {'__start__': '00000000000000000000000000000001.0.9568381425969369'}}, 'channel_values': {'user_age': 18, 'messages': [HumanMessage(content='你好，我叫Pet', additional_kwargs={}, response_metadata={}, id='b1578147-3bb4-48dc-9cfd-175288da9fa8'), AIMessage(content='你好，Pet！很高兴认识你！😊\n\nPet是个很特别的名字呢！有什么我可以帮助你的吗？无论是聊天、回答问题，还是需要一些建议，我都很乐意为你提供帮助。今天过得怎么样？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 8, 'total_tokens': 53, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_

#### get_state（graph）

graph.get_state()是高级方法，它内部调用get_tuple获取数据后，再包装成包含更多元信息的StateSnapshot对象返回。

In [70]:
with PostgresSaver.from_conn_string(db_uri) as checkpointer:
   
    agent = create_agent(
        model,
        state_schema=CustomState,
        checkpointer=checkpointer,
        debug=True,
    )
    state = agent.get_state(
        config={
            "configurable": {
                "thread_id": "example_thread_1",
                # "checkpoint_id": "",
            }
        }
    )
    print(state)


StateSnapshot(values={'messages': [HumanMessage(content='你好，我叫Pet', additional_kwargs={}, response_metadata={}, id='b1578147-3bb4-48dc-9cfd-175288da9fa8'), AIMessage(content='你好，Pet！很高兴认识你！😊\n\nPet是个很特别的名字呢！有什么我可以帮助你的吗？无论是聊天、回答问题，还是需要一些建议，我都很乐意为你提供帮助。今天过得怎么样？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 8, 'total_tokens': 53, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '3826d6d9-2371-4964-97a7-58517b4b934e', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c9ebb-4ac4-7b81-98eb-e7644e8799b1-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 8, 'output_tokens': 45, 'total_tokens': 53, 'input_token_details': {'cache_read': 0}, 'output_token_d

### 访问线程历史

让我们再继续与大模型一次对话，看看会发生什么。

In [71]:
with PostgresSaver.from_conn_string(db_uri) as checkpointer:
    checkpointer.setup()  
    
    agent = create_agent(
        model,
        state_schema=CustomState,
        checkpointer=checkpointer,
        debug=True,
    )
    result = agent.invoke(
        input={"messages": [HumanMessage(content="你好，我叫什么名字，多少岁了")]},
        config={
            "configurable": {
                "thread_id": "example_thread_1"
            }
        }
    )
    print(result["messages"][-1].content)


2026-02-27 18:54:27,249 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"


[values] {'messages': [HumanMessage(content='你好，我叫Pet', additional_kwargs={}, response_metadata={}, id='b1578147-3bb4-48dc-9cfd-175288da9fa8'), AIMessage(content='你好，Pet！很高兴认识你！😊\n\nPet是个很特别的名字呢！有什么我可以帮助你的吗？无论是聊天、回答问题，还是需要一些建议，我都很乐意为你提供帮助。今天过得怎么样？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 8, 'total_tokens': 53, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '3826d6d9-2371-4964-97a7-58517b4b934e', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c9ebb-4ac4-7b81-98eb-e7644e8799b1-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 8, 'output_tokens': 45, 'total_tokens': 53, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}

在多轮对话中，通过读取检查点的信息构建上下文，LLM已经记住了之前的对话内容，从而可以在后续的对话中保持上下文的连贯性。

#### list（checkpointer）

In [72]:
with PostgresSaver.from_conn_string(db_uri) as checkpointer:
    checkpointer.setup()  

    checkpoints = checkpointer.list(
        config={"configurable": {"thread_id": "example_thread_1"}}
    )

    print(list(checkpoints))

[CheckpointTuple(config={'configurable': {'thread_id': 'example_thread_1', 'checkpoint_ns': '', 'checkpoint_id': '1f113cab-128c-6b8e-8004-90eca254769b'}}, checkpoint={'v': 4, 'id': '1f113cab-128c-6b8e-8004-90eca254769b', 'ts': '2026-02-27T10:54:30.175810+00:00', 'versions_seen': {'model': {'branch:to:model': '00000000000000000000000000000005.0.5754805436264102'}, '__input__': {}, '__start__': {'__start__': '00000000000000000000000000000004.0.46450141192220384'}}, 'channel_values': {'user_age': 18, 'messages': [HumanMessage(content='你好，我叫Pet', additional_kwargs={}, response_metadata={}, id='b1578147-3bb4-48dc-9cfd-175288da9fa8'), AIMessage(content='你好，Pet！很高兴认识你！😊\n\nPet是个很特别的名字呢！有什么我可以帮助你的吗？无论是聊天、回答问题，还是需要一些建议，我都很乐意为你提供帮助。今天过得怎么样？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 8, 'total_tokens': 53, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hi

#### get_state_history（graph）

通过调用graph.get_state_history(config)获取指定线程的图执行完整的StateSnapshot对象列表，这些检查点会按时间顺序排列，其中最新的检查点（即最近的 StateSnapshot）位于列表的最前面。

In [73]:
with PostgresSaver.from_conn_string(db_uri) as checkpointer:
   
    agent = create_agent(
        model,
        state_schema=CustomState,
        checkpointer=checkpointer,
        debug=True,
    )
    states = list(agent.get_state_history(
        config={
            "configurable": {
                "thread_id": "example_thread_1",
                # "checkpoint_id": "",
            }
        }
    ))
    print(states)
    print(len(states))


[StateSnapshot(values={'messages': [HumanMessage(content='你好，我叫Pet', additional_kwargs={}, response_metadata={}, id='b1578147-3bb4-48dc-9cfd-175288da9fa8'), AIMessage(content='你好，Pet！很高兴认识你！😊\n\nPet是个很特别的名字呢！有什么我可以帮助你的吗？无论是聊天、回答问题，还是需要一些建议，我都很乐意为你提供帮助。今天过得怎么样？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 8, 'total_tokens': 53, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '3826d6d9-2371-4964-97a7-58517b4b934e', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c9ebb-4ac4-7b81-98eb-e7644e8799b1-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 8, 'output_tokens': 45, 'total_tokens': 53, 'input_token_details': {'cache_read': 0}, 'output_token_

In [75]:
for i,state in enumerate(states):
    if "user_age" in state.values.keys():
        print(i, state.values["user_age"])


0 18
1 18
2 18
3 18
4 18


虽然user_age一直存在在state中，但是调用model回复时，只会传递state["messages"]，因此在model中无法直接使用user_age。可以通过修改system_message或者手动插入user_age到messages中来实现上下文的增强。

## 修改检查点

利用`update_state`方法可以修改特定检查点中的state，该方法接受以下三个参数：

config
- config中应包含 thread_id，用于指定要更新的线程。如果仅传入 thread_id，则表示在该线程的当前状态上进行更新（或创建一个新分支）。
- 如果在 config 中额外包含 checkpoint_id 字段，则表示从该线程中指定的检查点派生出一个新分支（即“fork”该检查点）。

---
values
- 这些是要用来更新状态的值。
- 需要注意的是，这种更新的处理方式与图中任意节点产生的状态更新完全一致。
- 这意味着：如果图状态中的某些通道（channels）定义了归约函数（reducer），那么这些 values 会被传递给对应的 reducer 进行处理，而不是直接覆盖原值。
- 换句话说，update_state 不会自动覆盖所有通道的值，而仅对未定义 reducer 的通道执行直接覆盖；对于有 reducer 的通道，则会调用 reducer 来合并新旧值。

---
as_node

在调用 update_state 时，你还可以选择性地指定一个名为 as_node 的参数，默认为None，通常用"model"表示模型回复节点，用"tools"表示工具调用节点。
- 如果提供了该参数，此次状态更新将被视为由 as_node 节点所产生。
- 如果未提供 as_node，系统会自动将其设为最近一次更新该状态的节点（前提是该来源明确、无歧义）。

In [10]:
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables import RunnableConfig
from typing import Annotated
from typing_extensions import TypedDict
from operator import add
import os

db_uri = os.getenv("DB_URI")
if not db_uri:
    raise ValueError("DB_URI environment variable is not set")

class State(TypedDict):
    foo: int
    bar: Annotated[list[str], add]

def node_a(state: State):
    return {"foo": "1", "bar": ["a"]}

with PostgresSaver.from_conn_string(db_uri) as checkpointer:
    checkpointer.setup()  # 自动创建相关表结构

    workflow = StateGraph(State)
    workflow.add_node(node_a)
    workflow.add_edge(START, "node_a")
    workflow.add_edge("node_a", END)

    graph = workflow.compile(checkpointer=checkpointer)

    config: RunnableConfig = {"configurable": {"thread_id": "example_thread_5"}}

    result1 = graph.invoke(
        {"foo": "", "bar":[]},
        config,  
    )

    print(result1)

    graph.update_state(
        config,
        {"foo": "2", "bar": ["b"]},
    )


{'foo': '1', 'bar': ['a']}


In [12]:
with PostgresSaver.from_conn_string(db_uri) as checkpointer:
    checkpointer.setup()  # 自动创建相关表结构
    workflow = StateGraph(State)
    workflow.add_node(node_a)
    workflow.add_edge(START, "node_a")
    workflow.add_edge("node_a", END)

    graph = workflow.compile(checkpointer=checkpointer)

    state = graph.get_state(config)
    print(state.values)


{'foo': '2', 'bar': ['a', 'b']}


## 删除所有检查点

调用`checkpointer.delete_thread(thread_id)`即可删除指定线程的所有检查点。

# Time Travel

在处理基于模型决策的非确定性系统时，详细检查其决策过程非常有用：
- 理解推理过程：分析导致成功结果的步骤。
- 调试错误：确定错误发生的位置和原因。
- 探索替代方案：测试不同的路径以发现更好的解决方案。

LangGraph 提供了时间旅行（time travel）功能来支持这些用例。具体来说，你可以从之前的某个检查点恢复执行——既可以重播相同的状态，也可以修改状态以探索替代路径。在所有情况下，`恢复过去的执行都会在历史记录中产生一个新的分支`。

在 LangGraph 中使用时间旅行，通常有以下不足后：
1. 运行图：使用 invoke 或 stream 方法，配合初始输入来运行图。
2. 识别检查点：在现有线程中识别检查点：
    - 使用 get_state_history 方法检索特定 thread_id 的执行历史，并找到所需的 checkpoint_id。
    - 或者，在你希望执行暂停的节点之前设置一个中断（interrupt）。然后你可以找到记录到该中断为止的最新检查点。
3. 更新图状态（可选）：使用 update_state 方法在检查点修改图的状态，并从替代状态恢复执行。
4. 从检查点恢复执行：使用 invoke 或 stream 方法，输入为 None，并包含适当的 thread_id 和 checkpoint_id 的配置。

In [13]:
import uuid
# 导入 uuid 模块（虽然在当前代码中未实际使用，但通常用于生成唯一ID）

from typing_extensions import TypedDict, NotRequired
# 从 typing_extensions 导入 TypedDict (用于定义结构化字典) 和 NotRequired (表示字典中的键是可选的)

from langgraph.graph import StateGraph, START, END
# 从 langgraph 导入核心组件：StateGraph (状态图), START (起始节点), END (结束节点)

from langchain.chat_models import init_chat_model
# 从 langchain 导入用于初始化聊天模型的函数

from langgraph.checkpoint.memory import InMemorySaver
# 从 langgraph 导入内存检查点保存器，用于持久化状态（让流程可恢复）


# 定义工作流的状态结构
class State(TypedDict):
    topic: NotRequired[str]  # 主题字段（可选）：用于存储生成的笑话主题
    joke: NotRequired[str]   # 笑话字段（可选）：用于存储最终生成的笑话


def generate_topic(state: State):
    """LLM 调用函数，用于生成笑话的主题"""
    # 调用模型，请求一个有趣的笑话主题
    msg = model.invoke("给我一个有趣的笑话主题")
    # 返回包含主题的状态更新字典
    return {"topic": msg.content}


def write_joke(state: State):
    """LLM 调用函数，用于根据给定的主题编写笑话"""
    # 调用模型，传入上一步生成的主题，请求编写一个简短的笑话
    msg = model.invoke(f"写一个关于 {state['topic']} 的笑话")
    # 返回包含笑话的状态更新字典
    return {"joke": msg.content}


# --- 构建工作流 ---

# 创建一个 StateGraph 实例，传入之前定义的状态类
workflow = StateGraph(State)

# 添加节点（步骤）到图中
# 每个节点关联一个处理函数
workflow.add_node("generate_topic", generate_topic)
workflow.add_node("write_joke", write_joke)

# 添加边（连接）来定义执行顺序
workflow.add_edge(START, "generate_topic")  # 从开始指向 "generate_topic" 节点
workflow.add_edge("generate_topic", "write_joke")  # 从 "generate_topic" 指向 "write_joke" 节点
workflow.add_edge("write_joke", END)  # 从 "write_joke" 指向结束

# --- 编译与运行 ---

# 初始化检查点保存器，使用内存存储（适合开发和测试）
checkpointer = InMemorySaver()

# 编译图，传入检查点保存器
# 编译后的 graph 对象即可用于执行
graph = workflow.compile(checkpointer=checkpointer)

# 此时 graph 已准备好，可以通过调用 graph.invoke() 来运行工作流

先运行一遍图。

In [ ]:
config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
    }
}
state = graph.invoke({}, config)

print(state["topic"])
print()
print(state["joke"])

### 识别需要重放的检查点

状态是以逆时间顺序（从最新到最旧）返回的。`state.next`可以理解为当前state下，将要执行的节点。

In [18]:
states = list(graph.get_state_history(config))

for state in states:
    print(state.next)
    print(state.config["configurable"]["checkpoint_id"])
    print()
    


()
1f113d6b-0948-6f3a-8002-72fa03d9bdcf

('write_joke',)
1f113d6a-6817-64ae-8001-0c5f56143e20

('generate_topic',)
1f113d69-c136-646e-8000-28ca3d9d1562

('__start__',)
1f113d69-c134-6df8-bfff-424c1c0bfee4



In [24]:
######## 需要选择的检查点 #########
# This is the state before last (states are listed in chronological order)
selected_state = states[1]
print(selected_state.next)
print(selected_state.values)

('write_joke',)
{'topic': '当然！这里有几个能激发灵感的笑话主题，每个都自带一个例子，你可以根据这些主题自由发挥：\n\n---\n\n### **1. 动物界的“职场生活”**\n   * **例子**：  \n     为什么鲨鱼从不炒股？  \n     ——因为它们已经是大盘（大盘鸡）的股东了。（谐音梗预警）\n\n### **2. 科技产品的“人性化吐槽”**\n   * **例子**：  \n     Siri和导航软件的对话：  \n     Siri：“你总让我‘重新规划路线’，是不是在暗示我的人生该转弯了？”  \n     导航：“不，我只是在说你刚才错过了结婚典礼现场。”\n\n### **3. 历史人物的“现代生活梗”**\n   * **例子**：  \n     诸葛亮草船借箭时，突然收到短信：  \n     “曹操快递提醒您，您的十万支箭正在派送中，请保持手机畅通，并对船身进行喷火攻击以完成签收。”\n\n### **4. 食物界的“内部矛盾”**\n   * **例子**：  \n     面条和包子打架，包子输了。  \n     包子被揍成馅饼后哭着说：“没想到你是‘面’善心狠！”\n\n### **5. 超自然生物的“日常烦恼”**\n   * **例子**：  \n     吸血鬼最近很沮丧，朋友问他怎么了。  \n     他说：“现在年轻人熬夜比我还狠，我吸完血他们还能继续加班，显得我很不专业。”\n\n---\n\n### **进阶玩法：混搭主题**\n把两个主题结合，效果更炸裂：  \n**“AI机器人穿越到武侠世界”**  \n——少林寺方丈对扫地机器人说：“施主，你的‘轻功’（静音模式）虽好，但扫了三天，落叶为何一片未少？”  \n机器人：“方丈，我在进行深度学习，目前主要练习‘识别落叶’。”\n\n---\n\n选一个你最喜欢的主题，试着编一个吧！如果需要帮忙完善，随时告诉我 😄'}


### 状态更新（可选）

In [26]:
# 读取所选state的config，里面包含thread_id和checkpoint_id
# 更改值，这里将topic修改为chickens，返回的是RunnableConfig对象
new_config = graph.update_state(selected_state.config, values={"topic": "chickens"})
print(new_config)


{'configurable': {'thread_id': '7f25f230-9851-4c1c-86d2-2263896de7ae', 'checkpoint_ns': '', 'checkpoint_id': '1f113d8b-5a33-6222-8002-d953fc01f5b5'}}


### 重新执行

In [27]:
result = graph.invoke(None, new_config)
print(result["topic"])
print()
print(result["joke"])


2026-02-27 20:36:54,410 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"


chickens

为什么小鸡们总在吵架？

因为它们总在争论“先有鸡还是先有蛋”——但每次吵到一半，其中一只就会突然大喊：“等等！我们是不是忽略了重点？我们明明都会下蛋，但谁会煎蛋啊？”

另一只小鸡沉思两秒，拍翅膀说：“难怪人类总盯着我们看……他们是不是在等外送？”


### 案例：工具执行错误重放

在生产环境中，比较常见的一个场景是，当模型识别到需要调用一个工具时，会在消息中包含工具调用的相关信息，紧接着会执行工具调用返回tool_message，如果工具执行出现错误，缺失了tool_message，会导致在下次对话时报错，因为一但产生了tool_call，模型就需要依赖tool_message来继续执行，我们来模拟下这个场景。

In [ ]:
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
import time

@tool
def get_weather(location: str) -> str:
    """获取指定地点的天气。"""
    # 模拟等待 60 秒，方便终止工具调用
    time.sleep(60)
    return f"{location} 的天气是 20 摄氏度，晴朗。"


agent = create_agent(
    model=model,
    tools=[get_weather],
    checkpointer=InMemorySaver(),
    debug=True
)

config: RunnableConfig = {"configurable": {"thread_id": "thread_2"}}

result = agent.invoke(
    {"messages": [HumanMessage(content="宁波明天天气如何？")]},
    config=config
    
)
print(result)


In [48]:
states = agent.get_state_history(
    config=config
)
for state in states:
    print(state.next)
    print(state.tasks)
    print(state.config["configurable"]["checkpoint_id"])
    print() 

('model',)
(PregelTask(id='6e63227d-d9a2-b894-0131-84666cf88a35', name='model', path=('__pregel_pull', 'model'), error='BadRequestError(\'Error code: 400 - {\\\'error\\\': {\\\'message\\\': "An assistant message with \\\'tool_calls\\\' must be followed by tool messages responding to each \\\'tool_call_id\\\'. (insufficient tool messages following tool_calls message)", \\\'type\\\': \\\'invalid_request_error\\\', \\\'param\\\': None, \\\'code\\\': \\\'invalid_request_error\\\'}}\')', interrupts=(), state=None, result=None),)
1f113e96-d94c-6a16-8003-053a8cb1461b

('__start__',)
(PregelTask(id='03ba136a-af50-45f5-24df-436717ef3423', name='__start__', path=('__pregel_pull', '__start__'), error=None, interrupts=(), state=None, result={'messages': [HumanMessage(content='你好啊，介绍一下你自己', additional_kwargs={}, response_metadata={}, id='76a012bf-7e98-41d1-9c1c-84740a4f3b4e')]}),)
1f113e96-d947-6c64-8002-3dbbb68f7f2d

('tools',)
(PregelTask(id='4f2d981b-66e3-9e00-f60b-d2d3af087127', name='tools', p

In [41]:
result = agent.invoke(
    {"messages": [HumanMessage(content="你好啊，介绍一下你自己")]},
    config=config
    
)

2026-02-27 22:34:31,360 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 400 Bad Request"


[values] {'messages': [HumanMessage(content='宁波明天天气如何？', additional_kwargs={}, response_metadata={}, id='3bfb4210-b69a-47f1-8390-2ec99dd7d87e'), AIMessage(content='我来帮您查询宁波明天的天气情况。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 303, 'total_tokens': 355, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 256}, 'prompt_cache_hit_tokens': 256, 'prompt_cache_miss_tokens': 47}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '2fbbcb2f-d605-4434-9bf4-64cedb14c2a1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c9f7c-3551-79f0-bc2c-50d1f92d22ce-0', tool_calls=[{'name': 'get_weather', 'args': {'location': '宁波'}, 'id': 'call_00_uhnT3oUn6HKiRFTkywlC6n6n', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 303, 'output_tokens': 52, 'total_tokens': 355, 'inpu

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. (insufficient tool messages following tool_calls message)", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_request_error'}}

从报错信息`An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'`可以看到，tool_calls 必须紧跟着对应的tool messages才能完成整个响应流程。这时候就需要重新执行工具调用对用的checkpoint来补全调用堆栈。

In [58]:
states = agent.get_state_history(
    config=config
)

for state in states:
    if state.next == ("tools",):
        for task in state.tasks:    
            if task.name == "tools":
                if task.result is None or not task.result:
                    new_config = state.config
                    print(f"空工具调用config: {config}")


空工具调用config: {'configurable': {'thread_id': 'thread_2', 'checkpoint_ns': '', 'checkpoint_id': '1f113e7f-9645-6bf8-8001-f924a2b151d5'}}


In [59]:

result = agent.invoke(None,config=new_config)
print(result)


[values] {'messages': [HumanMessage(content='宁波明天天气如何？', additional_kwargs={}, response_metadata={}, id='3bfb4210-b69a-47f1-8390-2ec99dd7d87e'), AIMessage(content='我来帮您查询宁波明天的天气情况。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 303, 'total_tokens': 355, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 256}, 'prompt_cache_hit_tokens': 256, 'prompt_cache_miss_tokens': 47}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '2fbbcb2f-d605-4434-9bf4-64cedb14c2a1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c9f7c-3551-79f0-bc2c-50d1f92d22ce-0', tool_calls=[{'name': 'get_weather', 'args': {'location': '宁波'}, 'id': 'call_00_uhnT3oUn6HKiRFTkywlC6n6n', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 303, 'output_tokens': 52, 'total_tokens': 355, 'inpu

2026-02-27 23:00:20,272 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"


[updates] {'model': {'messages': [AIMessage(content='根据查询结果，宁波当前的天气是20摄氏度，晴朗。不过，我需要向您说明一下，我目前只能查询当前天气情况，无法获取明天的天气预报。\n\n如果您需要了解宁波明天的具体天气预报，建议您：\n1. 查看手机上的天气应用\n2. 访问中国天气网等专业气象网站\n3. 关注当地气象部门发布的天气预报\n\n当前宁波天气很好，温度适宜，适合外出活动。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 383, 'total_tokens': 471, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 320}, 'prompt_cache_hit_tokens': 320, 'prompt_cache_miss_tokens': 63}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'dcb268b3-a13b-4bb3-b6c0-efc8108fd8e2', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c9f9d-6bdf-70a1-b170-83ca8c1bddd9-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 383, 'output_tokens': 88, 'total_tokens': 471, 'input_token_details': {'cache_read': 320}, 'output_token_details': {}})]}}
[values

下面是一个可以用于生产环境检查thread下是否有未完成工具调用，并且能获取checkpoint_id和tool_call_id的函数，大家有需要可以直接使用。

In [ ]:
from langchain.agents import create_agent
from langchain.checkpoint.postgres.aio import AsyncPostgresSaver

async def get_incomplete_tool_checkpoint(
    thread_id: str,
) -> Optional[Dict[str, str]]:
    """检查线程是否有未完成的工具调用，返回需要恢复的 checkpoint_id 和 tool_call_id
    如果 next 是 ('tools',) 且 tasks 中 tools 的 result 为 None，说明有未完成的工具调用
    """
    try:
        async with AsyncPostgresSaver.from_conn_string(get_decoded_db_uri()) as checkpointer:
            config = {"configurable": {"thread_id": thread_id}}
            histoty_agent = create_agent(
                model=model,
                tools=[search_knowledge_base, web_search],
                system_prompt=get_system_prompt(),
                state_schema=BaseAgentState,
                context_schema=BaseContext,
                checkpointer=checkpointer,
            )
            async for snapshot in histoty_agent.aget_state_history(config):
                if snapshot.next == ("tools",):
                    has_empty_tool_result = False
                    tool_call_id = None
                    for task in snapshot.tasks:
                        if task.name == "tools":
                            if task.result is None or not task.result:
                                has_empty_tool_result = True
                                break

                    if has_empty_tool_result:
                        checkpoint_id = snapshot.config["configurable"]["checkpoint_id"]
                        messages = snapshot.values.get("messages", [])

                        for msg in messages:
                            if hasattr(msg, "tool_calls") and msg.tool_calls:
                                tool_call_id = msg.tool_calls[0].get("id")
                                break

                        return {
                            "checkpoint_id": checkpoint_id,
                            "tool_call_id": tool_call_id,
                        }

            return None

    except Exception as e:
        return None

# 持久化模式

LangGraph 支持三种持久化模式，您可以根据应用程序的需求在性能和数据一致性之间进行权衡。持久化级别越高，工作流执行的开销就越大。持久化模式按持久性从低到高依次如下：

- "exit"：仅在图执行退出时（无论是成功完成、发生错误，还是因中断而停止）才将变更持久化。该模式对长时间运行的图提供最佳性能，但不会保存中间状态，因此如果在执行过程中发生系统故障（例如进程崩溃），则无法恢复。
- "async"：`默认值`，在执行下一步的同时异步地将变更持久化。该模式在性能和持久性之间取得了良好平衡，但如果进程在执行过程中崩溃，仍存在少量检查点未能写入的风险。
- "sync"：在开始下一步之前同步地将变更持久化。该模式确保每个检查点都在继续执行前被写入，从而提供高持久性，但会带来一定的性能开销。

```python
graph.stream(
    {"input": "test"},
    durability="sync"
)
```

# 子图的持久化

当在编译父图时提供检查点存储器（checkpointer），并且子图在编译时传入了checkpointer=True，LangGraph会自动将该检查点存储器传播给子级子图。

> 自动命名空间隔离：LangGraph会自动为子图生成一个唯一的、嵌套的config。
> - 它会把子图的运行状态保存在父图状态的特定字段下。
> - 这样，子图觉得自己独占了一个 Checkpointer，但实际上它是被“托管”在父图的 Checkpointer 体系下的。

In [ ]:
# 编译父图传递checkpointer
graph = builder.compile(checkpointer=checkpointer)

# ============== 子图传入checkpointer参数 ==============
subgraph_builder = StateGraph(...)
subgraph = subgraph_builder.compile(checkpointer=True)

## 查看子图状态

若要查看子图的状态，可以指定subgraphs参数：`graph.get_state(config, subgraphs=True)`

> 子图状态只能在子图被中断时查看。一旦你恢复图的运行，你将无法再访问子图的状态。
> 这涉及到 LangGraph 的状态合并机制：
> - 当子图正在运行时：子图拥有独立的“小本本”（状态）。此时，你可以查看这个“小本本”。
> - 当子图运行结束（并恢复）：子图会把自己的计算结果合并回父图的状态中，然后子图自己的“小本本”通常会被清理或覆盖。